In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 21.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.7 MB/s eta 0:00

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import pandas as pd
path = '/content/labelled_tweets.csv'
encoder = preprocessing.LabelEncoder()
df = pd.read_csv(path)
df.rename(columns = {'label': 'labels'}, inplace=True)
#print(df.head())
encoder.fit(df['labels'])
df['labels']=encoder.transform(df['labels'])
print(df.head())
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['labels'], test_size=0.01, random_state=20)
train = pd.DataFrame([X_train, y_train]).T
train.set_index('tweet').dropna().to_csv('train.csv')

test = pd.DataFrame([X_test, y_test]).T
test.set_index('tweet').dropna().to_csv('test.csv')

                                               tweet  labels
0  In Nigeria, if you rape a woman, the men rape ...       0
1                            Then, she's a keeper. 😉       0
2  This is like the Metallica video where the poo...       0
3                                             woman?       0
4                     I bet she wished she had a gun       0


In [ ]:
from datasets import load_dataset, load_metric
import numpy as np
model_metric = load_metric('accuracy')

<ipython-input-3-0ac10517b774>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  model_metric = load_metric('accuracy')


In [ ]:
def compute_metric(model_pred):
  print(model_pred)
  preds, labels = model_pred
  # else: preds, labels, acc = model_pred
  preds = np.argmax(preds, -1)
  model_metric.compute(predictions=preds, references=labels)

In [ ]:
dataset = load_dataset("csv", data_files="train.csv")
test_set = load_dataset("csv", data_files="test.csv")
print(dataset['train']['tweet'][10])
set(dataset['train']['labels']), dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-96965208b5288dea/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-6afc2169aadd42a0/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Are you hate following Sinny? Might be why.


({0, 1},
 DatasetDict({
     train: Dataset({
         features: ['tweet', 'labels'],
         num_rows: 19800
     })
 }))

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest",num_labels = 2,ignore_mismatched_sizes=True)

def tokenize_fn(samples):
  return tokenizer(samples['tweet'], max_length=512, truncation=True, padding='max_length')

tokenized_train_set = dataset['train'].map(tokenize_fn, batched=True)
tokenized_test_set = test_set['train'].map(tokenize_fn, batched=True)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

Map:   0%|          | 0/19800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    label_names=['class'],
    fp16=True,
    save_strategy='no',
    do_eval=True,
    # eval_steps=10,
    logging_strategy="epoch"
)
# fp16 => For using GPU

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        loss = outputs.loss
        # print('----------------------------------', model(**inputs), '---------------------------------')
        return (loss, outputs) if return_outputs else loss


trainer = Trainer(model=model,
                  args=args,
                  train_dataset=tokenized_train_set,
                  eval_dataset=tokenized_test_set,
                  compute_metrics=compute_metric
                  )

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.372500,No log
2,0.279400,No log
3,0.189600,No log


TrainOutput(global_step=7425, training_loss=0.2804791732428451, metrics={'train_runtime': 2436.5939, 'train_samples_per_second': 24.378, 'train_steps_per_second': 3.047, 'total_flos': 1.5628796688384e+16, 'train_loss': 0.2804791732428451, 'epoch': 3.0})

In [ ]:
pred = trainer.predict(tokenized_test_set)
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
a = F.softmax(torch.tensor(pred.predictions[1], dtype=torch.float)).argmax(-1)
accuracy_score(a, y_test)

<ipython-input-9-28fc1102d561>:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(torch.tensor(pred.predictions[1], dtype=torch.float)).argmax(-1)


0.905

In [ ]:
trainer.save_model('Saved_model_ROBERTA')

In [ ]:
!zip -r /content/Saved_model_ROBERTA.zip /content/Saved_model_ROBERTA

  adding: content/Saved_model_ROBERTA/ (stored 0%)
  adding: content/Saved_model_ROBERTA/pytorch_model.bin (deflated 7%)
  adding: content/Saved_model_ROBERTA/config.json (deflated 49%)
  adding: content/Saved_model_ROBERTA/training_args.bin (deflated 48%)


In [ ]:
!mv '/content/Saved_model_ROBERTA.zip' '/content/drive/MyDrive'